# Google Drive API 연동 설정

이 노트북에서는 Google Drive API를 사용하여 RFP 문서를 자동으로 다운로드하는 설정을 진행합니다.

## 1. 패키지 import

In [1]:
import os
import sys
from pathlib import Path

# 프로젝트 루트를 Python path에 추가
project_root = Path().absolute().parent
sys.path.append(str(project_root))

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

ModuleNotFoundError: No module named 'google_auth_oauthlib'

## 2. Google Drive API 인증

### 사전 준비사항:
1. Google Cloud Console에서 프로젝트 생성
2. Google Drive API 활성화
3. OAuth 2.0 클라이언트 ID 생성 (데스크톱 앱)
4. `credentials.json` 파일을 프로젝트 루트에 저장

In [ ]:
# Google Drive API 스코프
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google_drive():
    """Google Drive API 인증"""
    creds = None
    token_path = project_root / 'token.json'
    credentials_path = project_root / 'credentials.json'
    
    # 기존 토큰 파일이 있으면 로드
    if token_path.exists():
        creds = Credentials.from_authorized_user_file(str(token_path), SCOPES)
    
    # 유효한 토큰이 없으면 새로 인증
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            if not credentials_path.exists():
                print("❌ credentials.json 파일이 없습니다!")
                print("   Google Cloud Console에서 OAuth 2.0 인증 정보를 다운로드하세요.")
                return None
            
            flow = InstalledAppFlow.from_client_secrets_file(
                str(credentials_path), SCOPES)
            creds = flow.run_local_server(port=0)
        
        # 토큰 저장
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    
    return creds

# 인증 실행
print("Google Drive 인증 중...")
creds = authenticate_google_drive()

if creds:
    print("✅ 인증 성공!")
    service = build('drive', 'v3', credentials=creds)
else:
    print("❌ 인증 실패")

## 3. Google Drive 폴더 탐색

데이터가 있는 폴더를 찾아봅니다.

In [ ]:
def list_files_in_folder(service, folder_name=None, folder_id=None):
    """
    Google Drive 폴더 내 파일 목록 조회
    
    Args:
        service: Google Drive API service
        folder_name: 폴더 이름 (검색용)
        folder_id: 폴더 ID (직접 지정)
    """
    try:
        if folder_id:
            # 폴더 ID로 직접 조회
            query = f"'{folder_id}' in parents and trashed=false"
        elif folder_name:
            # 폴더 이름으로 검색
            query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
            results = service.files().list(
                q=query,
                spaces='drive',
                fields='files(id, name)'
            ).execute()
            
            folders = results.get('files', [])
            if not folders:
                print(f"'{folder_name}' 폴더를 찾을 수 없습니다.")
                return []
            
            folder_id = folders[0]['id']
            print(f"폴더 찾음: {folder_name} (ID: {folder_id})")
            query = f"'{folder_id}' in parents and trashed=false"
        else:
            # 전체 파일 조회
            query = "trashed=false"
        
        # 파일 목록 조회
        results = service.files().list(
            q=query,
            spaces='drive',
            fields='files(id, name, mimeType, size)',
            pageSize=100
        ).execute()
        
        files = results.get('files', [])
        return files
    
    except Exception as e:
        print(f"에러 발생: {e}")
        return []

# 프로젝트 데이터 폴더 찾기
print("\n데이터 폴더를 찾고 있습니다...")
print("폴더 이름 또는 ID를 입력하세요:")

# 예시: "원본데이터" 폴더 검색
# files = list_files_in_folder(service, folder_name="원본데이터")

# 또는 직접 폴더 ID 입력
# files = list_files_in_folder(service, folder_id="YOUR_FOLDER_ID_HERE")

# 임시로 전체 파일 보기
files = list_files_in_folder(service)

print(f"\n찾은 파일 수: {len(files)}개")
for i, file in enumerate(files[:10], 1):  # 처음 10개만 출력
    size = int(file.get('size', 0)) / 1024 / 1024 if 'size' in file else 0
    print(f"{i}. {file['name']} ({size:.2f} MB)")

## 4. 파일 다운로드 함수

In [ ]:
def download_file(service, file_id, file_name, save_dir):
    """
    Google Drive에서 파일 다운로드
    
    Args:
        service: Google Drive API service
        file_id: 파일 ID
        file_name: 파일 이름
        save_dir: 저장 디렉토리
    """
    try:
        save_path = Path(save_dir) / file_name
        
        # 이미 파일이 있으면 스킵
        if save_path.exists():
            print(f"⏭️  스킵: {file_name} (이미 존재)")
            return True
        
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if status:
                progress = int(status.progress() * 100)
                print(f"\r⬇️  다운로드 중: {file_name} ({progress}%)", end="")
        
        # 파일 저장
        with open(save_path, 'wb') as f:
            f.write(fh.getvalue())
        
        print(f"\r✅ 완료: {file_name}")
        return True
    
    except Exception as e:
        print(f"\r❌ 실패: {file_name} - {e}")
        return False

# 테스트: 첫 번째 파일 다운로드
if files:
    test_file = files[0]
    save_dir = project_root / 'data' / 'raw'
    save_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\n테스트: {test_file['name']} 다운로드 중...")
    download_file(service, test_file['id'], test_file['name'], save_dir)

## 5. 다음 단계

✅ Google Drive API 인증 완료

다음 노트북: `01_data_download.ipynb`에서 전체 데이터 다운로드를 진행합니다.